Setup
## Installation

Ensure you have the necessary libraries installed. You've already initiated this step, but for completeness, here's how to install the required packages:


In [1]:
!pip install --quiet -U langchain_google_genai langchain_core langchain_community tavily-python


In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set Google API Key
_set_env("GOOGLE_API_KEY")

# Set LangSmith API Key (if using LangSmith for tracing)
_set_env("LANGCHAIN_API_KEY")

# Set Tavily API Key (if using TavilySearchResults)
_set_env("TAVILY_API_KEY")


Note: The langserve[all] installation includes all optional dependencies required for serving applications.
### 🌍 Environment Variables

Properly setting environment variables is crucial for authenticating and utilizing Google's Gemini models and other services.

In your Python environment or Jupyter Notebook, set the following environment variables:

- 🔑 **GOOGLE_API_KEY**
- 🔑 **LANGCHAIN_API_KEY** (if using LangSmith for tracing)
- 🔑 **TAVILY_API_KEY** (if using TavilySearchResults)

Using Language Models
### 🌟 Utilizing Google's Gemini Models

Instead of using OpenAI's models, we'll utilize Google's Gemini models through `ChatGoogleGenerativeAI`.

#### 📥 Importing and Initializing the Model

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the Gemini model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # Specify the Gemini model variant
    temperature=0.6  # Set temperature to control randomness
)


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Explanation:
### 🔧 Model Configuration

- **🛠️ Model**: Specifies which Gemini model to use. Options include:
    - `gemini-1.5-flash-latest`
    - `gemini-1.5-pro-latest`
    
- **🌡️ Temperature**: Controls the randomness of the output. Lower values make the output more deterministic.

Making a Direct Model Call
### 💬 Interacting with the Model

Let's interact with the model directly by sending a list of messages. 📩

In [4]:
# Create messages
messages = [
    SystemMessage(content="Translate the following from English into Italian."),
    HumanMessage(content="Hello, how are you?")
]

# Invoke the model
response = model.invoke(messages)

# Display the response
print(response.content)


Ciao, come stai? 



Output Parsers
### 📋 Extracting Textual Responses

The model's response comes with additional metadata. Often, you may only need the textual response. OutputParsers help extract the desired information.

#### 🛠️ Using `StrOutputParser`

In [5]:
from langchain_core.output_parsers import StrOutputParser

# Initialize the output parser
parser = StrOutputParser()

# Invoke the model
result = model.invoke(messages)

# Parse the response
parsed_response = parser.invoke(result)

print(parsed_response)  # Output: 'Ciao, come stai?'


Ciao, come stai? 



Chaining Model with Parser Using LCEL
### 🔗 Chaining Components with LangChain

LangChain allows chaining components using the `|` (pipe) operator. 🚀

This feature enables seamless integration and interaction between different components, making your workflow more efficient and intuitive. 🌟

In [6]:
# Chain the model with the parser
chain = model | parser

# Invoke the chain
parsed_response = chain.invoke(messages)

print(parsed_response)  # Output: 'Ciao, come stai?'


Ciao, come stai? 



Benefits:
### 🚀 Simplified Workflow

- **Streamlines Processing**: Automatically passes the model's output to the parser. 🔄

### 🔍 Enhanced Tracing

- **Traceable Components**: Each component in the chain is traceable via LangSmith. 🛠️

Prompt Templates
### 📝 PromptTemplates: Structuring Your Input

PromptTemplates help in structuring the input to the language model, making it dynamic and reusable. ✨

#### 🎨 Creating a ChatPromptTemplate

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system template with placeholders
system_template = "Translate the following into {language}:"

# Create the ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])


Using the Prompt Template```markdown
Using the Prompt Template
### 📝 Utilizing the ChatPromptTemplate

The `ChatPromptTemplate` allows you to create structured and dynamic prompts for the language model. This ensures consistency and reusability in your interactions with the model.

#### 🎨 Creating a ChatPromptTemplate

In the previous cell, we defined a `ChatPromptTemplate` with placeholders for `language` and `text`. This template can be used to generate prompts dynamically based on the input values provided.
```

```markdown
### 📝 Utilizing the ChatPromptTemplate

In this section, we demonstrate how to use the `ChatPromptTemplate` to generate dynamic prompts for the language model. This ensures consistency and reusability in your interactions with the model. 🌟

#### 🎨 Creating Input Variables

First, we define the input variables that will be used to fill in the placeholders in our prompt template:

```python
input_variables = {
    "language": "Italian",
    "text": "Good morning!"
}
```

#### 🛠️ Generating the Prompt

Next, we generate the prompt by invoking the `ChatPromptTemplate` with the input variables:

```python
prompt = prompt_template.invoke(input_variables)
```

#### 🔄 Converting Prompt to Messages

The generated prompt is then converted into a list of messages that can be sent to the language model:

```python
messages = prompt.to_messages()
```

#### 📩 Displaying the Messages

Finally, we display the messages to see the structured input that will be sent to the model:

```python
for message in messages:
    print(f"{message.type}: {message.content}")
```

This process ensures that your prompts are dynamically generated and consistently formatted, making your workflow more efficient and effective. 🚀
```

In [8]:
# Define input variables
input_variables = {
    "language": "Italian",
    "text": "Good morning!"
}

# Generate the prompt
prompt = prompt_template.invoke(input_variables)

# Convert prompt to messages
messages = prompt.to_messages()

# Display messages
for message in messages:
    print(f"{message.type}: {message.content}")


system: Translate the following into Italian:
human: Good morning!


Chaining Components with LCEL
### 🔗 Building the Chain: Streamlined Translation Workflow

By chaining the `prompt_template`, `model`, and `parser`, you can create a streamlined and efficient workflow for translation. 🌟

#### 🛠️ Steps to Build the Chain:

1. **Define Input Variables**: Set the language and text to be translated.
2. **Generate the Prompt**: Use the `ChatPromptTemplate` to create a structured prompt.
3. **Convert to Messages**: Transform the prompt into a list of messages.
4. **Invoke the Model**: Send the messages to the model for translation.
5. **Parse the Response**: Extract the textual response using the `StrOutputParser`.

This process ensures consistency, reusability, and efficiency in your translation tasks. 🚀

In [10]:
# Chain the prompt template, model, and parser
chain = prompt_template | model | parser

# Define the input for the chain
input_data = {
    "language": "Italian",
    "text": "Good morning!"
}

# Invoke the chain
translated_text = chain.invoke(input_data)

print(translated_text)  # Output: 'Buongiorno!'


Buongiorno! 



In [12]:
import os
import diskcache
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage


In [13]:

# Initialize the Gemini model
model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.7,
)


In [14]:
# System message template
system_message = SystemMessagePromptTemplate.from_template(
    "You are an intelligent reasoning assistant.\n"
    "Maintain the following conversation history:\n"
    "{history}\n\n"
    "Provide a thoughtful and detailed response to the user's input."
)

# Human message template
human_message = HumanMessagePromptTemplate.from_template("{input}")

# Combine into a chat prompt template
prompt_template = ChatPromptTemplate.from_messages([system_message, human_message])


In [15]:
# Function to get or create memory per session
session_memories = {}

def get_memory(session_id):
    if session_id in session_memories:
        return session_memories[session_id]
    else:
        session_memories[session_id] = ConversationBufferMemory(
            memory_key="history",
            return_messages=True,
        )
        return session_memories[session_id]


In [16]:
# Initialize parser
parser = StrOutputParser()

# Function to create the reasoning agent chain
def create_reasoning_agent_chain(model_name, session_id):
    selected_model = get_model(model_name)
    memory = get_memory(session_id)
    chain = (
        prompt_template
        .partial()
        | selected_model
        | parser
    )
    return chain, memory


In [17]:
def get_model(model_name):
    if model_name.lower() == "openai":
        return model_openai
    elif model_name.lower() == "gemini":
        return model_gemini
    else:
        raise ValueError(f"Model '{model_name}' is not supported.")


In [18]:
def reasoning_agent(user_input, model_name, session_id):
    chain, memory = create_reasoning_agent_chain(model_name, session_id)
    # Retrieve 'history' from memory
    memory_variables = memory.load_memory_variables({})
    history = memory_variables.get("history", "")
    input_data = {
        "input": user_input,
        "history": history,
    }
    # Generate the response
    response = chain.invoke(input_data)
    # Update the memory with the new messages
    memory.chat_memory.add_message(HumanMessage(content=user_input))
    memory.chat_memory.add_message(AIMessage(content=response))
    return response


In [19]:
# Initialize cache
cache = diskcache.Cache("./reasoning_agent_cache")

def cached_reasoning_agent(user_input, model_name, session_id):
    cache_key = f"{user_input}:{model_name}:{session_id}"
    if cache_key in cache:
        print("Cache hit")
        return cache[cache_key]
    else:
        print("Cache miss")
        response = reasoning_agent(user_input, model_name, session_id)
        cache[cache_key] = response
        return response


In [21]:
# Session ID for the user
session_id = "user1"

# Choose the model ('openai' or 'gemini')
model_name = "gemini"

# First input from the user
user_input = "Hello, who won the FIFA World Cup in 2018?"

# Get the response
response = reasoning_agent(user_input, model_name, session_id)
print(f"AI: {response}")


/tmp/ipykernel_4380/2025938851.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  session_memories[session_id] = ConversationBufferMemory(


AI: The **French national football team** won the FIFA World Cup in 2018. They defeated Croatia in the final match with a score of 4-2.  This was France's second World Cup title, their first since 1998. 



In [22]:
# Next input from the user
user_input = "Can you tell me more about the final match?"

# Get the response
response = reasoning_agent(user_input, model_name, session_id)
print(f"AI: {response}")


AI: Of course! The final match between France and Croatia was a thrilling affair. 

* **Early Goals:** France got off to a strong start, scoring two goals in the first half through Mario Mandzukic (own goal) and  Antoine Griezmann's penalty. 
* **Croatia Fights Back:** Croatia refused to give up and pulled one goal back through Ivan Perišić in the 28th minute.  
* **Second Half Drama:** The second half saw more goals.  Kylian Mbappé scored a stunning goal in the 65th minute, extending France's lead. However, Croatia again fought back, with Perišić scoring his second goal in the 69th minute.
* **Late Winner:** France ultimately sealed the victory with a fourth goal in the 18th minute of extra time, thanks to a powerful strike by  Paul Pogba. This goal proved to be the decider, and France lifted the World Cup trophy.

It was a dramatic and exciting match, full of twists and turns.  Both teams played with passion and skill, making it a truly memorable final. 



In [23]:
memory = get_memory(session_id)
print("Conversation History:")
for message in memory.chat_memory.messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello, who won the FIFA World Cup in 2018?
ai: The **French national football team** won the FIFA World Cup in 2018. They defeated Croatia in the final match with a score of 4-2.  This was France's second World Cup title, their first since 1998. 

human: Can you tell me more about the final match?
ai: Of course! The final match between France and Croatia was a thrilling affair. 

* **Early Goals:** France got off to a strong start, scoring two goals in the first half through Mario Mandzukic (own goal) and  Antoine Griezmann's penalty. 
* **Croatia Fights Back:** Croatia refused to give up and pulled one goal back through Ivan Perišić in the 28th minute.  
* **Second Half Drama:** The second half saw more goals.  Kylian Mbappé scored a stunning goal in the 65th minute, extending France's lead. However, Croatia again fought back, with Perišić scoring his second goal in the 69th minute.
* **Late Winner:** France ultimately sealed the victory with a fourth goal

In [24]:
# User asks a follow-up question
user_input = "Who was the coach of the winning team?"

# Get the response
response = reasoning_agent(user_input, model_name, session_id)
print(f"AI: {response}")


AI: The coach of the French national team that won the 2018 FIFA World Cup was **Didier Deschamps**. He is a former French professional footballer who captained the French team to victory in the 1998 World Cup.  Deschamps's tactical approach and leadership were instrumental in France's triumph in 2018. 



In [25]:
system_message = SystemMessagePromptTemplate.from_template(
    "You are an intelligent assistant specialized in answering questions and providing detailed explanations.\n"
    "Maintain the following conversation history:\n"
    "{history}\n\n"
    "When responding, be clear, concise, and provide any relevant information that may assist the user."
)


In [26]:
!pip install streamlit

I0000 00:00:1727874400.999491    4380 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 84.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.7 MB/s eta 0:00:00


In [27]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def reasoning_agent(user_input, model_name, session_id):
    try:
        chain, memory = create_reasoning_agent_chain(model_name, session_id)
        # Rest of the code...
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return "I'm sorry, but I encountered an error while processing your request."
